## Calibration data for pulse-echo ultrasound attenuation tomography

This notebook shows how to compute the calibration data for the pulse-echo ultrasound attenuation tomography technique presented in [tba]. Only linear probes are supported in this version.

Author: Naiara Korta Martiartu (naiara.korta@unibe.ch)\
Date: June 2023

In [ ]:
# Import useful packages

%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pickle
from attomo import Bmode, ATTomo

matplotlib.rcParams.update({'font.size': 18})

#### Loop over different realizations of acquired ultrasound signals in reference phantom

In [ ]:
num_real = 5  # number of realizations

# folder containing calibration data
folder_path = './data/'

# shared term in filenames
filename_root = 'calibration_acquisition_'

for i in range(0,num_real):    
    
    print(50 * '-')
    print(f'Realization #{i + 1}')
    print(50 * '-')
    
    ### Initialize Bmode object    
    rec = Bmode('kWave')
    
    
    ### Load data
    rec.load_data(folder_path, filename_root + f'{i + 1}.mat', use_filter=True) 
    
    
    ### STEP 1: Delay-and-sum beamforming
    rec.acq_delay = rec.acq_delay + 8.791438837681621e-07 # fix acquisition delay 
    rec.das(c = 1480, rec_range = 40e-3, gpu=True)
    
    
    ### STEP 2: Synthetic focusing
    
    # Define synthetic aperture angles
    start_saangle = -25  # [deg]
    end_saangle = 25
    delta_saangle = 2.5

    sa_angles = np.arange(start_saangle, end_saangle + delta_saangle, delta_saangle) * np.pi / 180  # [rad]

    # Define standard deviation for Gaussian weighting of images 
    sa_radius = 3
    sa_sigma=sa_radius*np.pi/180/np.sqrt(2)  # [rad]

    # Coherent compounding to reduce clutter 
    rec.coherent_compounding(sa_angles=sa_angles, sa_sigma=sa_sigma)
    
    
    ### STEP 3: Compute cross(auto)-correlations 
    # Initialize ATTomo object
    tomo = ATTomo(rec, spacing=[0.5e-3, 0.5e-3], gpu=True, range_rec=37.5e-3)
    
    # Measurments
    tomo.logamp_measurements(kernel_size=[1e-3, 1e-3])
    
    
    ### STEP 4: ensemble-average measurements
    if i==0:
        cc = tomo.ccs_map
        auto = tomo.auto_map
        auto2 = tomo.auto2_map
    else:
        cc += tomo.ccs_map  
        auto += tomo.auto_map
        auto2 += tomo.auto2_map
        
        
### STEP 5: Compute normalized cross-correlelation log-amplitudes    
tomo.amp_map = 0.5 * (np.log(np.abs(cc) / np.abs(auto), where=np.abs(cc) / np.abs(auto) > 0) -
                                  np.log(np.abs(cc) / np.abs(auto2), where=np.abs(cc) / np.abs(auto2) > 0))

#### Define relevant angles for tomography, add data and interpolate to tomography grid 

In [ ]:
start_angle = -25  # [deg]
end_angle = 25
delta_angle = 2.5

tomo_angles = np.arange(start_angle,end_angle + delta_angle,delta_angle)*np.pi/180  # [deg]


tomo.add_data(tomo_angles)

#### Save calibration data

In [ ]:
folder = './data/'
filename   = 'calibration.pkl'

with open(folder + filename,'wb') as f:  
    pickle.dump(tomo.tomo_logamp_map, f)

#### Plot calibration data

In [ ]:
fig = plt.figure(figsize=(15,12))

# Define extent of imaging domain
extent = 100 * np.min(tomo.tomo_x), 100 * np.max(tomo.tomo_x), 100 * np.max(tomo.tomo_z), 100 * np.min(tomo.tomo_z)

        
for i in range(tomo.tomo_nangles - 1):
    
    ax = fig.add_subplot(4, 5, i+1)    
    
    im = ax.imshow(tomo.tomo_logamp_map[:,:,i], aspect = 'equal', extent = extent,
                     vmin=-0.2, vmax=0.2, cmap='RdBu')
    
    
    
fig.colorbar(im)